# 局部线性嵌入（LLE）

## 概述

局部线性嵌入（Locally Linear Embedding, LLE）是一种非线性降维方法，属于流形学习算法。它假设数据分布在一个低维流形上，且每个数据点的局部邻域近似为线性结构。

## 核心思想

LLE 的核心假设是：每个数据点可以由其近邻点的线性组合来近似表示，且这种线性关系在降维后的低维空间中保持不变。

## 算法步骤

1. **寻找近邻**：对每个点 $x_i$，找到其 $K$ 个最近邻
2. **计算重构权重**：求解权重 $w_{ij}$ 使得 $x_i \approx \sum_{j \in \mathcal{N}(i)} w_{ij} x_j$
3. **低维嵌入**：在低维空间中寻找点 $y_i$，保持相同的线性重构关系

$$\min_Y \sum_i \left\| y_i - \sum_{j \in \mathcal{N}(i)} w_{ij} y_j \right\|^2$$

## 本节内容

1. LLE 基本使用
2. 瑞士卷数据的展开
3. n_neighbors 参数的影响
4. 与其他流形学习方法的比较

## 1. 环境准备

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import LocallyLinearEmbedding, Isomap, MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.datasets import make_swiss_roll, make_s_curve
import warnings

warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 2. 生成流形数据

使用经典的瑞士卷（Swiss Roll）和 S 曲线数据集。

In [ ]:
# 生成瑞士卷数据
n_samples = 1500
X_swiss, color_swiss = make_swiss_roll(n_samples=n_samples, noise=0.5, random_state=42)

# 生成 S 曲线数据
X_scurve, color_scurve = make_s_curve(n_samples=n_samples, noise=0.1, random_state=42)

print(f"瑞士卷数据形状: {X_swiss.shape}")
print(f"S 曲线数据形状: {X_scurve.shape}")

In [ ]:
# 3D 可视化
fig = plt.figure(figsize=(14, 5))

# 瑞士卷
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(X_swiss[:, 0], X_swiss[:, 1], X_swiss[:, 2], 
            c=color_swiss, cmap='viridis', s=10, alpha=0.8)
ax1.set_title('Swiss Roll')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.view_init(elev=10, azim=70)

# S 曲线
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(X_scurve[:, 0], X_scurve[:, 1], X_scurve[:, 2], 
            c=color_scurve, cmap='viridis', s=10, alpha=0.8)
ax2.set_title('S-Curve')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.view_init(elev=10, azim=70)

plt.tight_layout()
plt.show()

## 3. LLE 基本使用

将 3D 瑞士卷数据展开到 2D 平面。

In [ ]:
# 创建 LLE 模型
lle = LocallyLinearEmbedding(
    n_components=2,      # 降维到 2 维
    n_neighbors=12,      # 每个点的近邻数
    random_state=42
)

# 拟合并转换数据
X_lle = lle.fit_transform(X_swiss)

print(f"原始数据形状: {X_swiss.shape}")
print(f"降维后数据形状: {X_lle.shape}")
print(f"重构误差: {lle.reconstruction_error_:.6f}")

In [ ]:
# 对比 PCA 和 LLE 的降维效果
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_swiss)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PCA 结果
scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], 
                           c=color_swiss, cmap='viridis', s=10, alpha=0.8)
axes[0].set_title('PCA (Linear)')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')

# LLE 结果
scatter2 = axes[1].scatter(X_lle[:, 0], X_lle[:, 1], 
                           c=color_swiss, cmap='viridis', s=10, alpha=0.8)
axes[1].set_title('LLE (Nonlinear)')
axes[1].set_xlabel('LLE1')
axes[1].set_ylabel('LLE2')

plt.colorbar(scatter2, ax=axes[1], label='Position on manifold')
plt.suptitle('Unfolding the Swiss Roll: PCA vs LLE', fontsize=14)
plt.tight_layout()
plt.show()

## 4. n_neighbors 参数的影响

`n_neighbors` 是 LLE 最重要的超参数：
- **太小**：流形可能"断裂"，无法捕捉全局结构
- **太大**：趋近于线性方法，失去非线性展开能力

In [ ]:
# 测试不同的 n_neighbors 值
n_neighbors_values = [5, 10, 20, 50, 100, 200]

fig, axes = plt.subplots(2, 3, figsize=(15, 9))

for ax, n_neighbors in zip(axes.flat, n_neighbors_values):
    lle = LocallyLinearEmbedding(n_components=2, n_neighbors=n_neighbors, random_state=42)
    
    try:
        X_lle = lle.fit_transform(X_swiss)
        ax.scatter(X_lle[:, 0], X_lle[:, 1], 
                   c=color_swiss, cmap='viridis', s=10, alpha=0.8)
        ax.set_title(f'n_neighbors = {n_neighbors}\nerror = {lle.reconstruction_error_:.4f}')
    except Exception as e:
        ax.text(0.5, 0.5, f'Failed\n{str(e)[:30]}', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'n_neighbors = {n_neighbors}')
    
    ax.set_xlabel('LLE1')
    ax.set_ylabel('LLE2')

plt.suptitle('Effect of n_neighbors Parameter on LLE', fontsize=14)
plt.tight_layout()
plt.show()

## 5. LLE 的不同变体

scikit-learn 提供了几种 LLE 变体：
- **standard**：标准 LLE
- **hessian**：Hessian LLE，更稳定
- **modified**：改进的 LLE，处理稀疏邻域
- **ltsa**：局部切空间对齐

In [ ]:
# 测试不同的 LLE 变体
methods = ['standard', 'hessian', 'modified', 'ltsa']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, method in zip(axes.flat, methods):
    # hessian 方法需要更多近邻
    n_neighbors = 15 if method != 'hessian' else 20
    
    lle = LocallyLinearEmbedding(
        n_components=2, 
        n_neighbors=n_neighbors,
        method=method,
        random_state=42
    )
    
    try:
        X_lle = lle.fit_transform(X_swiss)
        ax.scatter(X_lle[:, 0], X_lle[:, 1], 
                   c=color_swiss, cmap='viridis', s=10, alpha=0.8)
        ax.set_title(f'{method.upper()} LLE\nerror = {lle.reconstruction_error_:.4f}')
    except Exception as e:
        ax.text(0.5, 0.5, f'Failed', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'{method.upper()} LLE')
    
    ax.set_xlabel('Dim 1')
    ax.set_ylabel('Dim 2')

plt.suptitle('Different LLE Variants on Swiss Roll', fontsize=14)
plt.tight_layout()
plt.show()

## 6. 与其他流形学习方法比较

比较 LLE、Isomap、MDS 和 t-SNE 在瑞士卷上的表现。

In [ ]:
# 使用较少的样本以加速 t-SNE
n_subset = 1000
indices = np.random.choice(n_samples, n_subset, replace=False)
X_subset = X_swiss[indices]
color_subset = color_swiss[indices]

# 定义不同的方法
methods = [
    ('PCA', PCA(n_components=2)),
    ('LLE', LocallyLinearEmbedding(n_components=2, n_neighbors=12, random_state=42)),
    ('Isomap', Isomap(n_components=2, n_neighbors=12)),
    ('MDS', MDS(n_components=2, random_state=42, normalized_stress='auto')),
    ('t-SNE', TSNE(n_components=2, perplexity=30, random_state=42))
]

fig = plt.figure(figsize=(15, 9))

# 原始 3D 数据
ax_3d = fig.add_subplot(2, 3, 1, projection='3d')
ax_3d.scatter(X_subset[:, 0], X_subset[:, 1], X_subset[:, 2], 
              c=color_subset, cmap='viridis', s=10, alpha=0.8)
ax_3d.set_title('Original 3D Data')
ax_3d.view_init(elev=10, azim=70)

# 各种降维方法
for i, (name, method) in enumerate(methods):
    ax = fig.add_subplot(2, 3, i + 2)
    X_transformed = method.fit_transform(X_subset)
    ax.scatter(X_transformed[:, 0], X_transformed[:, 1], 
               c=color_subset, cmap='viridis', s=10, alpha=0.8)
    ax.set_title(name)
    ax.set_xlabel('Dim 1')
    ax.set_ylabel('Dim 2')

plt.suptitle('Comparison of Manifold Learning Methods', fontsize=14)
plt.tight_layout()
plt.show()

## 7. S 曲线数据的展开

In [ ]:
# 在 S 曲线上比较 PCA 和 LLE
pca = PCA(n_components=2)
X_pca_s = pca.fit_transform(X_scurve)

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=12, random_state=42)
X_lle_s = lle.fit_transform(X_scurve)

fig = plt.figure(figsize=(15, 4))

# 原始 3D 数据
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(X_scurve[:, 0], X_scurve[:, 1], X_scurve[:, 2], 
            c=color_scurve, cmap='viridis', s=10, alpha=0.8)
ax1.set_title('Original S-Curve (3D)')
ax1.view_init(elev=10, azim=70)

# PCA 结果
ax2 = fig.add_subplot(132)
ax2.scatter(X_pca_s[:, 0], X_pca_s[:, 1], 
            c=color_scurve, cmap='viridis', s=10, alpha=0.8)
ax2.set_title('PCA')
ax2.set_xlabel('PC1')
ax2.set_ylabel('PC2')

# LLE 结果
ax3 = fig.add_subplot(133)
scatter = ax3.scatter(X_lle_s[:, 0], X_lle_s[:, 1], 
                      c=color_scurve, cmap='viridis', s=10, alpha=0.8)
ax3.set_title('LLE')
ax3.set_xlabel('LLE1')
ax3.set_ylabel('LLE2')

plt.colorbar(scatter, ax=ax3, label='Position')
plt.suptitle('Unfolding S-Curve', fontsize=14)
plt.tight_layout()
plt.show()

## 8. 总结

### 关键要点

1. **核心思想**：
   - 保持数据的局部线性结构
   - 适合展开分布在流形上的数据

2. **n_neighbors 选择**：
   - 典型值：10-20
   - 太小：流形断裂，结果不连续
   - 太大：趋向线性降维，失去非线性特性

3. **LLE 变体**：
   - **Standard**：最基本，快速
   - **Hessian**：更稳定，需要更多近邻
   - **Modified**：处理稀疏邻域更好
   - **LTSA**：使用局部切空间

4. **与其他方法比较**：
   - **vs PCA**：LLE 能展开非线性流形
   - **vs Isomap**：LLE 更关注局部结构
   - **vs t-SNE**：LLE 更注重保持流形的全局结构

### 适用场景

- 数据分布在低维流形上
- 需要保持局部邻域关系
- 可视化高维数据的内在结构

### 注意事项

- 对噪声敏感
- 不支持新样本的转换（需要重新训练）
- 计算复杂度 $O(n^2)$，不适合大规模数据